In [2]:
# loading Game
import gym
#env = gym.make('Pong-ram-v0')
env = gym.make('Pong-v0')
env.reset()

# Setting training parmaters
max_frames = 300
score_requirement = -2
inital_games = 10


##env = gym.make('CartPole-v0')
##max_frames = 500
##score_requirement = 50
##inital_games = 10000



In [ ]:
def play_game():
    for ep in range(inital_games):
        env.reset()
        for t in range(200):
            env.render()
            
            action = str(input())
            if action == "q":
                env.close()
                return 
            
            action = 2 if action == "w" else 5 if action =="s" else 0
            
            observation, reward, done, info = env.step(action)
            
            print("T: ",t,"\tAction : ",ACTION_MEANING[action])
            if done:
                break
 


In [ ]:
play_game()

In [ ]:
env.close()

In [24]:
def random_action():
    r = random.randrange(0,2)
    if r == 0:
        return UP
    elif r == 1:
        return DOWN

def tobits(act):
    if act == 2:
        # action was up
        return 1
    elif act == 5:
        # action was down
        return -1
    else:
        # action was noop
        return 0

In [25]:
# test the set of action of the game one frame at a time
# enter 'w' for up, 'd' for down and enter for no op

#play_game()

def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    
    for episode in range(inital_games):
        score = 0
        game_memory = []
        observation = env.reset()

        for _ in range(max_frames):
            # playing an game until max frames reached or game is done
            
#             env.render() ## un comment to run the game
            action = random_action()
            game_memory.append([observation,tobits(action)])
            
            observation, reward, done, info = env.step(action)

            
            score += reward
            
            if done:
                break

        print("=> Episode #",episode+1,"\tScore: ",score)
        
        if score >= score_requirement:
            accepted_scores.append(score)
            
            for data in game_memory:                
                training_data.append([data[0],data[1]])
                
        scores.append(score)
    
#     training_data_save = np.array(training_data)
#     np.save('saved_R.npy',training_data_save)
        
#     print("Accepted score: ",accepted_scores)
    
#     print("\nAverage scores: ",mean(scores))
#     print("Average accepted score: ",mean(accepted_scores))
#     print("Median accepted score: ",median(accepted_scores))
#     print(Counter(accepted_scores))
    env.close()

    return training_data

In [7]:
Dataset = initial_population()

=> Episode # 1 	Score:  -5.0
=> Episode # 2 	Score:  -3.0
=> Episode # 3 	Score:  -3.0
=> Episode # 4 	Score:  -2.0
=> Episode # 5 	Score:  -2.0
=> Episode # 6 	Score:  -4.0
=> Episode # 7 	Score:  -5.0
=> Episode # 8 	Score:  -2.0
=> Episode # 9 	Score:  -5.0
=> Episode # 10 	Score:  -4.0


In [85]:
env.close()

In [8]:
print("Size of dataset: ",len(Dataset))
Dataset[1]

Size of dataset:  900


[array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [144,  72,  17],
         [144,  72,  17],
         [144,  72,  17]],
 
        [[144,  72,  17],
         [144,  72,  17],
         [144,  72,  17],
         ...,
         [144,  72,  17],
         [144,  72,  17],
         [144,  72,  17]],
 
        [[144,  72,  17],
         [144,  72,  17],
         [144,  72,  17],
         ...,
         [144,  72,  17],
         [144,  72,  17],
         [144,  72,  17]],
 
        ...,
 
        [[236, 236, 236],
         [236, 236, 236],
         [236, 236, 236],
         ...,
         [236, 236, 236],
         [236, 236, 236],
         [236, 236, 236]],
 
        [[236, 236, 236],
         [236, 236, 236],
         [236, 236, 236],
         ...,
         [236, 236, 236],
         [236, 236, 236],
         [236, 236, 236]],
 
        [[236, 236, 236],
         [236, 236, 236],
         [236, 236, 236],
         ...,
         [236, 236, 236],
  

In [9]:
def process_data(datum):
    downsample = np.mean(datum[:, ::2, ::2], axis=3)
    return downsample.reshape(*downsample.shape, 1)

In [10]:
model = Sequential()
model.add(Conv2D(256, (3, 3), input_shape=(105, 80, 1), padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1, activation="linear", kernel_initializer="uniform"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
X_train = np.array([i[0] for i in Dataset])

def f(x):
    return x if x == 1 else 0

y_train = np.array([f(i[1]) for i in Dataset])
y_train = y_train.reshape(y_train.shape[0], 1)
print(y_train.shape)
print(X_train[0].shape)
model.summary()

(900, 1)
(210, 160, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 105, 80, 256)      2560      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 40, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 532480)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 532481    
Total params: 535,041
Trainable params: 535,041
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(process_data(X_train), y_train, epochs=20)

Epoch 1/20
900/900 [==============================] - 24s 27ms/step - loss: 8.0952 - acc: 0.0000e+00
Epoch 2/20
900/900 [==============================] - 24s 26ms/step - loss: 8.0952 - acc: 0.0000e+00
Epoch 3/20
900/900 [==============================] - 26s 29ms/step - loss: 8.0952 - acc: 0.0000e+00
Epoch 4/20
288/900 [========>.....................] - ETA: 23s - loss: 8.1926 - acc: 0.0000e+00

KeyboardInterrupt: 

In [21]:
process_data(X_train[:2])

array([[[[  0.        ],
         [  0.        ],
         [  0.        ],
         ...,
         [ 90.        ],
         [ 90.        ],
         [ 90.        ]],

        [[ 90.        ],
         [ 90.        ],
         [ 90.        ],
         ...,
         [ 90.        ],
         [ 90.        ],
         [ 90.        ]],

        [[ 90.        ],
         [ 90.        ],
         [ 90.        ],
         ...,
         [ 90.        ],
         [ 90.        ],
         [ 90.        ]],

        ...,

        [[ 57.33333333],
         [ 57.33333333],
         [ 57.33333333],
         ...,
         [ 57.33333333],
         [ 57.33333333],
         [ 57.33333333]],

        [[ 57.33333333],
         [ 57.33333333],
         [ 57.33333333],
         ...,
         [ 57.33333333],
         [ 57.33333333],
         [ 57.33333333]],

        [[ 57.33333333],
         [ 57.33333333],
         [ 57.33333333],
         ...,
         [ 57.33333333],
         [ 57.33333333],
         [ 57.333

In [22]:
model.predict(process_data(X_train[:2]))

array([[116.371925],
       [106.324295]], dtype=float32)

# work from saturday

In [ ]:
## From satudary

In [ ]:
def random_game():
    for epsiode in range(5):
        print("========== Episode #",epsiode+1," ==========")
        env.reset()
        for t in range(max_frames):
            env.render()
            
            a = str(input())
            if a == 'q':
                env.close()
                return
            elif a == 'n':
                break
                
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            
            print("T: ",t,"\tAction : ",ACTION_MEANING[action])

            
                
            if done:
                break

                


def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    
    for episode in range(inital_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(max_frames):
            ##env.render()
            action = Pong_random_action()
            #action = env.action_space.sample()
            observation, reward, done, info = env.step(action)

            if len(prev_observation) > 0:
                game_memory.append([prev_observation,action])
            
            prev_observation = observation
            
            score += reward
            
            if done:
                break

        print("=> Episode #",episode+1,"\tScore: ",score)
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                output = None
                if data[1] == 2:# action up
                    output = 1
                if data[1] == 5: # action down
                    output = 0
                    
                training_data.append([data[0],output])
                
        env.reset()
        scores.append(score)
    
    training_data_save = np.array(training_data)
    np.save('saved_R.npy',training_data_save)
        
    print("Accepted score: ",accepted_scores)
    
    print("\nAverage scores: ",mean(scores))
    print("Average accepted score: ",mean(accepted_scores))
    print("Median accepted score: ",median(accepted_scores))
    print(Counter(accepted_scores))

    return training_data
        
        
        
        
        

In [ ]:
training_data_set = initial_population()

In [ ]:
print("Length of training data: ",len(training_data_set))
training_data_set = np.array(training_data_set)
np.array(training_data_set[0][0]).shape


In [ ]:
print("Training dataset: ",np.array(training_data_set).shape)
print("Target: ",env.observation_space.shape)


X_train = np.array([i[0] for i in training_data_set])
y_train = np.array([i[1] for i in training_data_set])


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten  

In [ ]:
# defines the model
model = Sequential()
model.add(Dense(128, input_shape=env.observation_space.shape, activation="relu", kernel_initializer="uniform"))
model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
model.add(Dense(64, activation="relu", kernel_initializer="uniform"))
model.add(Dense(2, activation="linear", kernel_initializer="uniform"))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])


In [ ]:
# training sets


model.fit([X_train], y_train, epochs=10, batch_size=32,verbose=1)

#training_data_set[0][0].shape

**Uncomment the cell bellow to save the model**

In [ ]:
#from keras.models import load_model

#model.save("FirstModel_withRandom.h5")

#model = load_model('FirstModel_withRandom.h5')

yuoo = None

In [ ]:
def binary_to_action(bit):
    if bit == [0,1]:
        return 5
    else:
        return 2

def game_with_model():
    scores = []
    choices = []
    for each_game in range(10):
        score = 0
        game_memory = []
        prev_obs = []
        env.reset()
        for _ in range(max_frames):
            env.render()
            if len(prev_obs) == 0:
                action = Pong_random_action()
            else:
                #predict based on our model
                #print("obs shape: ",prev_obs.shape)
                #print(prev_obs)
                action = model.predict([[prev_obs]])
            
            choices.append(action)
            observation, reward, done, info = env.step(action)
            prev_obs = observation
            game_memory.append([observation,action])
            score += reward
            
            if done:
                break
        scores.append(score)
        
    print("\nAverage scores: ",mean(scores))
    print("Chose UP ",choices.count(2)/len(choices)," chose Down ",choices.count(5)/len(choices))


In [ ]:
game_with_model()

In [ ]:
env.close()

In [ ]:
model.predict(yuoo)


# Run the cell before the code

In [6]:
# standard modules 
import numpy as np
import random 
from collections import Counter
from statistics import mean, median

# keras import
from keras.models import Sequential
from keras.layers import Dense, Flatten  
from keras.models import load_model
from keras.layers.convolutional import Conv2D, MaxPooling2D

# list of actions
ACTION_MEANING = {
    0 : "NOOP",
    1 : "FIRE",
    2 : "UP",
    3 : "RIGHT",
    4 : "LEFT",
    5 : "DOWN",
    6 : "UPRIGHT",
    7 : "UPLEFT",
    8 : "DOWNRIGHT",
    9 : "DOWNLEFT",
    10 : "UPFIRE",
    11 : "RIGHTFIRE",
    12 : "LEFTFIRE",
    13 : "DOWNFIRE",
    14 : "UPRIGHTFIRE",
    15 : "UPLEFTFIRE",
    16 : "DOWNRIGHTFIRE",
    17 : "DOWNLEFTFIRE",
}
UP = 2
DOWN = 5


## function that randomly returns up or down
def Pong_random_action():
    r = random.randrange(0,2)
    if r == 0:
        return UP
    else:
        return DOWN


Using TensorFlow backend.
